## Trabajo Fin de Grado
### Gestor de Quirófanos
### Implementación de Algoritmo Particle Swarm para optimización del orden de las intervenciones quirúrgicas

#### Autor: Jesús García Armario

In [2]:
# Imports necesarios
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd
import sys
sys.path.append('../')
from Heuristicas.Utils import Quirofano, ActoQuirurgico

In [3]:
# Importamos el listado preprocesado de una especialidad para las pruebas
filename = '..\\..\\Datos\\Listados_Preprocesados\\ListadoInterv_Preprocesado_MAXILOFACIAL.xlsx'
df = pd.read_excel(filename)
# Seleccionamos una muestra aleatoria de 100 pacientes
df = df.sample(n=100, random_state=1)
# Nos quedamos sólo con el NHC, Ponderación y duración
df = df[['NHC', 'PONDERACIÓN', 'DURACIÓN']]
# Sumamos a la duración 25 minutos por paciente para tener en cuenta el tiempo de preparación
df['DURACIÓN'] = df['DURACIÓN'] + 25
ventana = 30
# Dividimos la duración entre la ventana y redondeamos hacia arriba
df['DURACIÓN'] = df['DURACIÓN'].apply(lambda x: math.ceil(x/ventana))
# Creamos un set de actos quirúrgicos
actos_pendientes = list()
i = 0
for elemento in df.itertuples():
    actos_pendientes.append(ActoQuirurgico(i, elemento[3], elemento[1], elemento[2]))
    i += 1

In [4]:
# Creamos 100 partículas válidas
def crear_particula(tiempos, quirofanos, dias, actos_pendientes):
    actos = actos_pendientes.copy()
    particula = []
    # Creamos una particula aleatoria
    for i in range(dias):
        for j in range(quirofanos):
            T = tiempos
            while T > 0:
                if len(actos) == 0:
                    particula.append('V')
                    T -= 1
                else:
                    acto = random.choice(actos)
                    intentos = 0
                    while acto.getDuracion() > T:
                        acto = random.choice(actos)
                        intentos += 1
                        if intentos > len(actos):
                            particula.append('V')
                            T -= 1
                            break
                    particula.append(acto.getId())
                    actos.remove(acto)
                    T -= acto.getDuracion()
            particula.append('A')
        particula.append('B')
    return particula

# Creamos una población de 100 partículas válidas
def crear_poblacion(tiempos, quirofanos, dias, actos_pendientes, n):
    poblacion = []
    for i in range(n):
        poblacion.append(crear_particula(tiempos, quirofanos, dias, actos_pendientes))
    return poblacion

poblacion = crear_poblacion(tiempos = 16, quirofanos = 3, dias = 5, actos_pendientes = actos_pendientes, n = 100)